# TF-IDF and Encoder-LSTM, LIME Aggregator

### Install Necessary Libraries

In [ ]:
#pip install torchtext
#!pip install transformers
#!pip install torchsummary 

In [1]:
from gensim.utils import simple_preprocess
import torch
import torch.optim as optim
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from torch import nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader, TensorDataset, WeightedRandomSampler
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import Vocab
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
nltk.download('punkt')
nltk.download('stopwords')
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
%matplotlib inline
from collections import Counter
from tqdm import tqdm
from torch.utils.data.dataset import random_split
from scipy.sparse import csr_matrix
from transformers import BertModel
import shap
from lime.lime_text import LimeTextExplainer
import torch.nn.functional as F

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


2024-04-22 23:05:34.640065: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-22 23:05:34.640164: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-22 23:05:34.774962: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Load Twitter Dataset, Clean and Select Features for Models

In [2]:
df = pd.read_csv("/kaggle/input/twitter-data-virality/dataset_viralscore_2.csv", low_memory=False)
df['UserFavouritesCount'] = pd.to_numeric(df['UserFavouritesCount'], errors='coerce')
df = df.dropna(subset=['UserFavouritesCount'])
df['mediaCount'] = pd.to_numeric(df['mediaCount'], errors='coerce')
df = df.dropna(subset=['mediaCount'])
df = df.sample(frac=1).reset_index(drop=True)
df

,url,date,content,likeCount,replyCount,retweetCount,viewCount,quoteCount,sourceLabel,links,...,tokenized,cleaned_text,likeCount_log,replyCount_log,viewCount_log,totalpop,userpopularity,totalScore,ViralGroup,ViralGroupAdjusted
0,https://twitter.com/pannchoa/status/1589407272...,2022-11-06 23:59:58+00:00,Red Velvet releases The ReVe Festival 2022 tea...,889.0,16.0,178,58702.620000,51,Twitter Web App,1,...,"['Red', 'Velvet', 'releases', 'The', 'ReVe', '...",Red Velvet releases The ReVe Festival teaser i...,0.509429,0.298540,0.544019,143505.0,High,1.351988,High,High
1,https://twitter.com/ovokkj/status/154447107448...,2022-07-05 23:59:33+00:00,diazin https://t.co/XVx1NQOiwT,11.0,5.0,0,1202.246025,1,Twitter for Android,0,...,['diazin'],diazin,0.186400,0.188800,0.174161,1003.0,Low,0.549361,Low,High
2,https://twitter.com/milkstrology/status/156766...,2022-09-07 23:59:41+00:00,venus in virgo squaring mars in gemini reminds...,667.0,6.0,45,26079.540000,24,Twitter Web App,0,...,"['venus', 'virgo', 'squaring', 'mars', 'gemini...",venus virgo squaring mars gemini reminds hyper...,0.487906,0.205043,0.466830,390932.0,High,1.159779,High,High
3,https://twitter.com/TxQueenTee/status/15017093...,2022-03-09 23:59:41+00:00,now that page is attacking me and y’all wanna ...,38.0,11.0,1,3240.231232,0,Twitter for iPhone,0,...,"['page', 'attacking', 'wanna', 'know', 'whats'...",page attacking wanna know whats funny screensh...,0.274814,0.261838,0.268439,5228.0,Low,0.805091,High,High
4,https://twitter.com/minarchist9/status/1511856...,2022-04-06 23:59:46+00:00,@folding_laundry not a bad take,1.0,0.0,0,191.910011,0,Twitter Web App,0,...,"['folding_laundry', 'bad', 'take']",folding_laundry bad take,0.051995,0.000000,0.000000,406.0,Low,0.051995,Low,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43787,https://twitter.com/makeitslowerr/status/15256...,2022-05-14 23:59:40+00:00,what they’re about to do is literally life alt...,16.0,1.0,0,1902.447461,0,Twitter for iPhone,0,...,"['literally', 'life', 'altering', 'magic', 'fe...",literally life altering magic feel ready terri...,0.212528,0.073038,0.217797,1870.0,Low,0.503362,Low,High
43788,https://twitter.com/fuckinbiscuit/status/15676...,2022-09-07 23:59:57+00:00,@emersonRpage idk what’s up with my luck fr bu...,0.0,0.0,0,210.006559,0,Twitter for iPhone,0,...,"['emersonRpage', 'idk', 'luck', 'fr', 'somehow...",emersonRpage idk luck fr somehow every health ...,0.000000,0.000000,0.008531,558.0,Low,0.008531,Low,Low
43789,https://twitter.com/m_sarr/status/157418688694...,2022-09-25 23:59:35+00:00,How they expect Melissa not to date? #BlackWi...,2.0,0.0,0,302.782800,0,Twitter for iPhone,0,...,"['How', 'expect', 'Melissa', 'date', 'BlackWid...",How expect Melissa date BlackWidowMurders,0.082410,0.000000,0.043203,2736.0,Low,0.125613,Low,Low
43790,https://twitter.com/LukeThisisnot/status/15491...,2022-07-18 23:59:50+00:00,Seems like a fitting end to this Monday. https...,16.0,4.0,0,1902.447461,0,Twitter for Android,0,...,"['Seems', 'like', 'fitting', 'end', 'Monday']",Seems like fitting end Monday,0.212528,0.169589,0.217797,1936.0,Low,0.599913,Low,High


In [3]:
df.groupby('ViralGroupAdjusted').size()

ViralGroupAdjusted
High    21896
Low     21896
dtype: int64

In [4]:
select_columns = ['links','media','quotedTweet','mentionedUsers','UserFavouritesCount','followersCount','friendsCount','verified','protected','mediaCount','predicted_sentiment']
#select_columns = ['links','media','quotedTweet','mentionedUsers','UserFavouritesCount','friendsCount','protected','mediaCount']

In [5]:
df_data = df[select_columns].copy()
df_data

,links,media,quotedTweet,mentionedUsers,UserFavouritesCount,followersCount,friendsCount,verified,protected,mediaCount,predicted_sentiment
0,1,1,0,0,5641,143474.0,31,0,0,14731.0,Neutral
1,0,1,0,0,18108,293.0,710,0,0,259.0,Neutral
2,0,0,0,0,71117,389770.0,1162,1,0,8760.0,Negative
3,0,0,0,0,82866,2854.0,2374,0,0,11232.0,Positive
4,0,0,0,1,1893,79.0,327,0,0,90.0,Negative
...,...,...,...,...,...,...,...,...,...,...,...
43787,0,1,0,0,123217,1141.0,729,0,0,4673.0,Positive
43788,0,0,0,0,18710,292.0,266,0,0,177.0,Negative
43789,0,1,0,0,88313,1630.0,1106,0,0,21350.0,Neutral
43790,0,1,0,0,17084,1293.0,643,0,0,2092.0,Neutral


In [6]:
sentiment_mapping = {'Negative': -1, 'Neutral': 0, 'Positive': 1}
df_data['predicted_sentiment'] = df_data['predicted_sentiment'].map(sentiment_mapping)

In [7]:
df_data

,links,media,quotedTweet,mentionedUsers,UserFavouritesCount,followersCount,friendsCount,verified,protected,mediaCount,predicted_sentiment
0,1,1,0,0,5641,143474.0,31,0,0,14731.0,0
1,0,1,0,0,18108,293.0,710,0,0,259.0,0
2,0,0,0,0,71117,389770.0,1162,1,0,8760.0,-1
3,0,0,0,0,82866,2854.0,2374,0,0,11232.0,1
4,0,0,0,1,1893,79.0,327,0,0,90.0,-1
...,...,...,...,...,...,...,...,...,...,...,...
43787,0,1,0,0,123217,1141.0,729,0,0,4673.0,1
43788,0,0,0,0,18710,292.0,266,0,0,177.0,-1
43789,0,1,0,0,88313,1630.0,1106,0,0,21350.0,0
43790,0,1,0,0,17084,1293.0,643,0,0,2092.0,0


In [8]:
print(df_data.dtypes)

links                    int64
media                    int64
quotedTweet              int64
mentionedUsers           int64
UserFavouritesCount      int64
followersCount         float64
friendsCount             int64
verified                 int64
protected                int64
mediaCount             float64
predicted_sentiment      int64
dtype: object


In [9]:
for col in df_data.columns:
    if df_data[col].dtype == np.object_:
        # Attempt to convert to float; handle exceptions as needed
        df_data[col] = df_data[col].astype(float)

In [10]:
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df_data), columns=df_data.columns)
df_scaled

,links,media,quotedTweet,mentionedUsers,UserFavouritesCount,followersCount,friendsCount,verified,protected,mediaCount,predicted_sentiment
0,2.985327,1.247817,-0.458066,-0.419791,-0.537304,0.002232,-0.219794,-0.375237,0.0,0.265445,-0.049500
1,-0.334972,1.247817,-0.458066,-0.419791,-0.406681,-0.090949,-0.171491,-0.375237,0.0,-0.207742,-0.049500
2,-0.334972,-0.801399,-0.458066,-0.419791,0.148718,0.162518,-0.139337,2.664982,0.0,0.070213,-1.339798
3,-0.334972,-0.801399,-0.458066,-0.419791,0.271818,-0.089282,-0.053117,-0.375237,0.0,0.151039,1.240799
4,-0.334972,-0.801399,-0.458066,2.382140,-0.576573,-0.091088,-0.198737,-0.375237,0.0,-0.213268,-1.339798
...,...,...,...,...,...,...,...,...,...,...,...
43787,-0.334972,1.247817,-0.458066,-0.419791,0.694594,-0.090397,-0.170140,-0.375237,0.0,-0.063419,1.240799
43788,-0.334972,-0.801399,-0.458066,-0.419791,-0.400374,-0.090949,-0.203077,-0.375237,0.0,-0.210424,-1.339798
43789,-0.334972,1.247817,-0.458066,-0.419791,0.328889,-0.090079,-0.143320,-0.375237,0.0,0.481865,-0.049500
43790,-0.334972,1.247817,-0.458066,-0.419791,-0.417410,-0.090298,-0.176257,-0.375237,0.0,-0.147809,-0.049500


## Generate Other Features as Tensors

In [12]:
df_sample = df_data.copy()
df_sample['ViralGroup'] = df['ViralGroup']
print(df_sample.head(10))

   links  media  quotedTweet  mentionedUsers  UserFavouritesCount  \
0      1      1            0               0                 5641   
1      0      1            0               0                18108   
2      0      0            0               0                71117   
3      0      0            0               0                82866   
4      0      0            0               1                 1893   
5      0      0            0               0                 5351   
6      0      1            0               1                29471   
7      0      1            0               0                43758   
8      0      0            1               0                20790   
9      0      0            0               1                30810   

   followersCount  friendsCount  verified  protected  mediaCount  \
0        143474.0            31         0          0     14731.0   
1           293.0           710         0          0       259.0   
2        389770.0          1162     

In [13]:
df_content = pd.DataFrame(df['content'])
df_content

,content
0,Red Velvet releases The ReVe Festival 2022 tea...
1,diazin https://t.co/XVx1NQOiwT
2,venus in virgo squaring mars in gemini reminds...
3,now that page is attacking me and y’all wanna ...
4,@folding_laundry not a bad take
...,...
43787,what they’re about to do is literally life alt...
43788,@emersonRpage idk what’s up with my luck fr bu...
43789,How they expect Melissa not to date? #BlackWi...
43790,Seems like a fitting end to this Monday. https...


In [14]:
def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove other non-word characters, numbers, etc., if you wish
    text = re.sub(r'\W+|\d+', ' ', text)
    return text

In [15]:
df_content['clean_content'] = df_content['content'].apply(clean_text)

In [16]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
df_content['tokenized'] = df_content['clean_content'].apply(lambda x: [word for word in x.split() if word not in (stop)])

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
df_content['cleaned_text'] = df_content['tokenized'].apply(lambda x: ' '.join(x))

In [18]:
vectorizer = TfidfVectorizer(min_df=0.001, max_df=0.7, smooth_idf=True)
tfidf_matrix = vectorizer.fit_transform(df_content['cleaned_text'])

In [19]:
tfidf_dense = tfidf_matrix.todense()
scaler2 = StandardScaler()
scaled_tfidf = scaler2.fit_transform(np.asarray(tfidf_dense))

In [20]:
tfidf_tensor = torch.tensor(scaled_tfidf, dtype=torch.float32)

In [21]:
features = torch.tensor(df_scaled.values, dtype=torch.float32)

In [22]:
combined_features = torch.cat((features, tfidf_tensor), dim=1)

### Encode Virality Labels and Generate Target Tensor for Training

In [23]:
mapping = {'High': 0, 'Low': 1}
df['ViralGroup_encoded'] = df['ViralGroupAdjusted'].map(mapping)

In [24]:
virals = ['ViralGroupAdjusted','ViralGroup_encoded']
df[virals]

,ViralGroupAdjusted,ViralGroup_encoded
0,High,0
1,High,0
2,High,0
3,High,0
4,Low,1
...,...,...
43787,High,0
43788,Low,1
43789,Low,1
43790,High,0


In [25]:
targets = torch.tensor(df['ViralGroup_encoded'].values)
targets = targets.long()

## TF-IDF

### Define Neural Network for TF-IDF Model

In [ ]:
class Model(nn.Module):
    def __init__(self,vocab_size, feature_size, num_classes):
        super(Model, self).__init__()
        self.fc_tfidf = nn.Linear(vocab_size, 16)
        self.relu0 = nn.ReLU()
        
        #self.fc1 = nn.Linear(feature_size+16, 64)  # 10 input features
        self.fc1 = nn.Linear(16, 64)  # 10 input features
        
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(64, 64)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(64, num_classes)   # 3 output classes

    def forward(self, tfidf, features):
        tfidf_out = self.fc_tfidf(tfidf)
        tfidf_out = self.relu0(tfidf_out)
        
        #combined = torch.cat((tfidf_out, features), 1)
        combined = tfidf_out
        
        x = self.fc1(combined)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)  # No activation, raw scores
        return x

In [ ]:
input_size = combined_features.shape[1]
vocab_size = tfidf_tensor.shape[1]
model = Model(vocab_size, input_size, num_classes=2)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

### Define Dataset and Loaders for Training Loop

In [ ]:
dataset = TensorDataset(tfidf_tensor, combined_features, targets)

class_counts = [0, 0]  #(Low, High)
for _, _, label in dataset:
    #print(label)
    class_counts[label] += 1

#weights = [0] * len(dataset)
#for idx, (_, _, label) in enumerate(dataset):
#    weights[idx] = 1.0 / class_counts[label]
    
# Define the size of train and test sets
train_size = int(0.8 * len(dataset))  # e.g., 80% of the dataset
test_size = len(dataset) - train_size  # The remaining 20% of the dataset

# Randomly split the dataset into train and test sets
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create weights for the entire dataset
full_weights = [1.0 / class_counts[label] for _,_, label in dataset]
# Adjust weights for train_dataset after split
train_weights = [full_weights[idx] for idx in train_dataset.indices]
test_weights = [full_weights[idx] for idx in test_dataset.indices]

In [ ]:
train_sampler = WeightedRandomSampler(train_weights, num_samples=len(train_dataset), replacement=True)
test_sampler = WeightedRandomSampler(test_weights, num_samples=len(test_dataset), replacement=True)

train_loader = DataLoader(train_dataset, batch_size=32, sampler=train_sampler)
test_loader = DataLoader(test_dataset, batch_size=32, sampler=test_sampler)

### Train TF-IDF Neutal Netword Model

In [ ]:
num_epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for tfidf, feature, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        tfidf, feature, labels = tfidf.to(device), feature.to(device), labels.to(device)  # Move to GPU if available
        optimizer.zero_grad()  # Zero the parameter gradients
        outputs = model(tfidf, feature)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights
        
        # Statistics
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()
        
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = correct_predictions / total_predictions
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")

In [ ]:
model.eval()
val_loss = 0.0
correct_val = 0
total_val = 0
n_classes = 2

# Initialize all possible class labels
class_correct = {i: 0 for i in range(n_classes)}
class_total = {i: 0 for i in range(n_classes)}
false_positives = {i: 0 for i in range(n_classes)}
false_negatives = {i: 0 for i in range(n_classes)}


with torch.no_grad():
    for tfidf, feature, labels in tqdm(test_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        tfidf, feature, labels = tfidf.to(device), feature.to(device), labels.to(device)  # Move to GPU if available

        # Forward pass
        outputs = model(tfidf, feature)
        loss = criterion(outputs, labels)

        # Calculate validation loss
        val_loss += loss.item()

        # Calculate validation accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_val += labels.size(0)
        correct_val += (predicted == labels).sum().item()

        # Update metrics
        for i in range(len(labels)):
            label = labels[i].item()
            pred = predicted[i].item()

            if label not in class_correct:
                class_correct[label] = 0
                class_total[label] = 0
                false_positives[label] = 0
                false_negatives[label] = 0

            if pred == label:
                class_correct[label] += 1
            else:
                false_positives[pred] += 1
                false_negatives[label] += 1

            class_total[label] += 1
        
val_loss /= len(test_loader)
val_accuracy = correct_val / total_val

overall_precision = sum(class_correct.values()) / (sum(class_correct.values()) + sum(false_positives.values()))
overall_recall = sum(class_correct.values()) / (sum(class_correct.values()) + sum(false_negatives.values()))
overall_f1 = 2 * (overall_precision * overall_recall) / (overall_precision + overall_recall) if (overall_precision + overall_recall) > 0 else 0


print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
print(f"Overall Precision: {overall_precision:.4f}, Overall Recall: {overall_recall:.4f}, Overall F1: {overall_f1:.4f}")

# Print accuracy for each class
for class_id in class_correct:
    class_acc = 100 * class_correct[class_id] / class_total[class_id]
    print(f"Accuracy of Class {class_id}: {class_acc:.2f}%")
    
# Print accuracy, precision, and recall for each class
for class_id in class_correct:
    accuracy = class_correct[class_id] / class_total[class_id] if class_total[class_id] > 0 else 0
    precision = class_correct[class_id] / (class_correct[class_id] + false_positives[class_id]) if class_correct[class_id] + false_positives[class_id] > 0 else 0
    recall = class_correct[class_id] / (class_correct[class_id] + false_negatives[class_id]) if class_correct[class_id] + false_negatives[class_id] > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    print(f"Class {class_id} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

## LSTM

### Setup BERT Tokenizer

In [26]:
from transformers import BertTokenizer

In [27]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_length = 128  # Choose a max_length that suits your dataset

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [28]:
def preprocess_text(text):
    text = re.sub(r'http\S+', '[URL]', text)
    text = re.sub(r'@\w+', '[MENTION]', text)
    
    tokens = tokenizer(text, max_length=max_length, padding='max_length', truncation=True, return_tensors="pt")
    return tokens['input_ids'], tokens['attention_mask']

In [29]:
special_tokens_dict = {'additional_special_tokens': ['[URL]', '[MENTION]']}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

In [30]:
processed_text = [preprocess_text(text) for text in df_content['content']]

In [31]:
input_ids = [item[0] for item in processed_text]
attention_masks = [item[1] for item in processed_text]

input_ids = torch.stack(input_ids).squeeze(1)
attention_masks = torch.stack(attention_masks).squeeze(1)

In [33]:
print("input_ids shape:", input_ids.shape)
print("attention_masks shape:", attention_masks.shape)
print("features shape:", features.shape)
print("targets shape:", targets.shape)

input_ids shape: torch.Size([43792, 128])
attention_masks shape: torch.Size([43792, 128])
features shape: torch.Size([43792, 11])
targets shape: torch.Size([43792])


### Define Dataset and Loader based on if doing Text-Only or All-Features

In [34]:
dataset = TensorDataset(input_ids, attention_masks, features, targets)
#dataset = TensorDataset(input_ids, attention_masks, combined_features, targets)

In [35]:
class_counts = [0, 0]  #(Low, High)
for _, _, _, label in dataset:
    #print(label)
    class_counts[label] += 1

#weights = [0] * len(dataset)
#for idx, (_, _, label) in enumerate(dataset):
#    weights[idx] = 1.0 / class_counts[label]
    
# Define the size of train and test sets
train_size = int(0.8 * len(dataset))  # e.g., 80% of the dataset
test_size = len(dataset) - train_size  # The remaining 20% of the dataset

# Randomly split the dataset into train and test sets
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create weights for the entire dataset
full_weights = [1.0 / class_counts[label] for _,_,_, label in dataset]
# Adjust weights for train_dataset after split
train_weights = [full_weights[idx] for idx in train_dataset.indices]
test_weights = [full_weights[idx] for idx in test_dataset.indices]

In [36]:
train_sampler = WeightedRandomSampler(train_weights, num_samples=len(train_dataset), replacement=True)
test_sampler = WeightedRandomSampler(test_weights, num_samples=len(test_dataset), replacement=True)

train_loader = DataLoader(train_dataset, batch_size=32, sampler=train_sampler)
test_loader = DataLoader(test_dataset, batch_size=32, sampler=test_sampler)

### Define and Initialize Model

Model For All Features

In [ ]:
class MyModel(nn.Module):
    def __init__(self, bert_model_name, num_other_features, lstm_hidden_size, lstm_layers, num_classes):
        super(MyModel, self).__init__()
        # BERT Model
        self.bert = BertModel.from_pretrained(bert_model_name)
        bert_output_size = self.bert.config.hidden_size
        # LSTM layer
        self.lstm = nn.LSTM(input_size=bert_output_size, hidden_size=lstm_hidden_size, 
                            num_layers=lstm_layers, batch_first=True)
        # Fully connected layers for combined features
        self.classifier = nn.Sequential(
            nn.Linear(lstm_hidden_size + num_other_features, 64),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(64, num_classes)
        )

    def forward(self, input_ids, attention_mask, other_features):
        # Handling text data with BERT
        with torch.no_grad():
            bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        bert_embeddings = bert_output.last_hidden_state

        # Passing BERT embeddings to LSTM
        lstm_output, _ = self.lstm(bert_embeddings)
        # Using only the last hidden state of LSTM
        lstm_output = lstm_output[:, -1, :]

        # Concatenating LSTM output with other features
        combined_features = torch.cat((lstm_output, other_features), dim=1)

        # Classifier
        output = self.classifier(combined_features)
        return output

Model for Text-Only

In [37]:
class MyModel2(nn.Module):
    def __init__(self, bert_model_name, lstm_hidden_size, lstm_layers, num_classes):
        super(MyModel2, self).__init__()
        # BERT Model
        self.bert = BertModel.from_pretrained(bert_model_name)
        bert_output_size = self.bert.config.hidden_size
        # LSTM layer
        self.lstm = nn.LSTM(input_size=bert_output_size, hidden_size=lstm_hidden_size, 
                            num_layers=lstm_layers, batch_first=True)
        # Fully connected layers for combined features
        self.classifier = nn.Sequential(
            nn.Linear(lstm_hidden_size, 64),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(64, num_classes)
        )

    def forward(self, input_ids, attention_mask):
        # Handling text data with BERT
        with torch.no_grad():
            bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        bert_embeddings = bert_output.last_hidden_state

        # Passing BERT embeddings to LSTM
        lstm_output, _ = self.lstm(bert_embeddings)
        # Using only the last hidden state of LSTM
        lstm_output = lstm_output[:, -1, :]

        # Concatenating LSTM output with other features
        #combined_features = torch.cat((lstm_output), dim=1)

        # Classifier
        output = self.classifier(lstm_output)
        return output

In [38]:
bert_model_name = 'bert-base-uncased'
num_other_features = features.shape[1]
#num_other_features = combined_features.shape[1]
lstm_hidden_size = 32
lstm_layers = 2
num_classes = 2

In [39]:
#lstm_model = MyModel(bert_model_name, num_other_features, lstm_hidden_size, lstm_layers, num_classes)
lstm_model = MyModel2(bert_model_name, lstm_hidden_size, lstm_layers, num_classes)
lstm_model.bert.resize_token_embeddings(len(tokenizer))

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Embedding(30524, 768)

In [40]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(lstm_model.parameters(), lr=0.001)

In [41]:
best_loss = np.inf

### Training Loop, Adjust Epoch Number or Run Multiple Times

In [ ]:
num_epochs = 1 # Number of epochs
n_iterations = 1000  # Number of iterations/batches per epoch
batch_size = 32  # Your batch size

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lstm_model = lstm_model.to(device)

for epoch in range(num_epochs):
    lstm_model.train()
    running_loss = 0.0
    batch_count = 0
    correct_predictions = 0
    total_predictions = 0

    with tqdm(total=n_iterations, desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch') as pbar:
        for i, batch in enumerate(train_loader):
            #print("Batch ",i)
            if i >= n_iterations:
                break  # Stop after n_iterations
            b_input_ids, b_attention_masks, b_other_features, b_labels = [t.to(device) for t in batch]
            optimizer.zero_grad()  # Zero the parameter gradients
            #outputs = lstm_model(b_input_ids, b_attention_masks, b_other_features)  # Forward pass
            outputs = lstm_model(b_input_ids, b_attention_masks)  # Forward pass
            loss = criterion(outputs, b_labels)  # Compute loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update weights

            # Statistics
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_predictions += b_labels.size(0)
            correct_predictions += (predicted == b_labels).sum().item()

            # Update tqdm bar
            pbar.update(1)  # This updates the progress bar by one iteration
            
            batch_count += 1
            if batch_count >= n_iterations:
                break
        
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = correct_predictions / total_predictions
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")

### Testing Loop. Update Best Model if Testing Loss decreases

In [ ]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lstm_model = lstm_model.to(device)
lstm_model.eval()  # Set the model to evaluation mode

total_loss = 0.0
all_labels = []
all_predictions = []
n_classes=num_classes

with torch.no_grad():  # No need to track gradients during evaluation
    for batch in tqdm(test_loader, desc="Evaluating", unit="batch"):
        b_input_ids, b_attention_masks, b_other_features, b_labels = [t.to(device) for t in batch]

        # Forward pass
        #outputs = lstm_model(b_input_ids, b_attention_masks, b_other_features)
        outputs = lstm_model(b_input_ids, b_attention_masks)

        # Compute loss
        loss = criterion(outputs, b_labels)
        total_loss += loss.item()

        _, predicted = torch.max(outputs, 1)

        all_labels.extend(b_labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

# Compute overall metrics
overall_accuracy = accuracy_score(all_labels, all_predictions)
overall_loss = total_loss / len(test_loader)
overall_precision = precision_score(all_labels, all_predictions)
overall_recall = recall_score(all_labels, all_predictions)
overall_f1 = f1_score(all_labels, all_predictions)

# Compute class-wise metrics
class_accuracies = []
precisions = []
recalls = []
f1_scores = []

for i in range(n_classes):
    labels_i = np.array(all_labels) == i
    predictions_i = np.array(all_predictions) == i

    class_accuracies.append(accuracy_score(labels_i, predictions_i))
    precisions.append(precision_score(labels_i, predictions_i, zero_division=0))
    recalls.append(recall_score(labels_i, predictions_i, zero_division=0))
    f1_scores.append(f1_score(labels_i, predictions_i, zero_division=0))

# Display results
print(f"Overall Loss: {overall_loss:.4f}, Overall Accuracy: {overall_accuracy:.4f}")
print(f"Overall Precision: {overall_precision:.4f}, Overall Recall: {overall_recall:.4f}, Overall F1: {overall_f1:.4f}")
print(f"Class Accuracies: {class_accuracies}")
print(f"Precision per Class: {precisions}")
print(f"Recall per Class: {recalls}")
print(f"F1 Score per Class: {f1_scores}")

#Update Best Model
if overall_loss < best_loss:
    print("Updating Best Model")
    best_model = lstm_model
    best_loss = overall_loss

In [ ]:
lstm_model = best_model

In [ ]:
best_model = lstm_model
best_loss = overall_loss

In [ ]:
torch.save(lstm_model, '/kaggle/working/model.pth')

In [ ]:
lstm_model = torch.load('/kaggle/input/twitter-data-virality/model.pth')

## LIME

### Use LIME to get word importance for each input

In [ ]:
device = "cuda"
lstm_model.to(device)
lstm_model.eval()

In [ ]:
def predict_proba(texts, batch_size=32):
    # List to hold all probabilities
    all_probabilities = []
    processed_texts = []
    
    for text in texts:
        text = re.sub(r'http\S+', '[URL]', text)  # Replace URLs with '[URL]'
        text = re.sub(r'@\w+', '[MENTION]', text)  # Replace mentions with '[MENTION]'
        processed_texts.append(text)
    
    # Process texts in batches
    for i in range(0, len(processed_texts), batch_size):
        batch_texts = processed_texts[i:i + batch_size]
        inputs = tokenizer(batch_texts, max_length=max_length, padding='max_length', truncation=True, return_tensors="pt")

        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']

        # Move to device
        device = next(lstm_model.parameters()).device
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)

        # Model prediction
        with torch.no_grad():
            logits = lstm_model(input_ids, attention_mask)

        # Convert logits to probabilities
        probabilities = torch.nn.functional.softmax(logits, dim=-1).cpu().numpy()
        all_probabilities.extend(probabilities)

        # Memory cleanup
        del input_ids, attention_mask, logits
        torch.cuda.empty_cache()
    all_probabilities = np.array(all_probabilities)
    return all_probabilities

In [ ]:
# Create an instance of LimeTextExplainer
explainer = LimeTextExplainer(class_names=['High', 'Low'])  # Customize class names based on your specific case

In [ ]:
random_row = df.sample(n=1)
r_content = random_row['content'].iloc[0]
random_row

In [ ]:
r_input_id, r_attention = preprocess_text(r_content)
lstm_model.eval()
r_input_id, r_attention = r_input_id.to(device), r_attention.to(device)
r_outputs = lstm_model(r_input_id, r_attention)
_, r_predicted = torch.max(r_outputs, 1)
print('Outputs: ',r_outputs)
print('Prediction: ',r_predicted)

In [ ]:
logits = r_outputs
probabilities = torch.nn.functional.softmax(logits, dim=-1)
print(probabilities)

In [ ]:
sample_text = r_content

In [ ]:
explanation = explainer.explain_instance(sample_text, predict_proba, num_features=20, num_samples=500)

# Visualize the explanation
explanation.show_in_notebook(text=True)

## Aggregated LIME

In [ ]:
from collections import defaultdict
from itertools import islice
import nltk


num_iterations = min(1000, len(df['content']))

In [ ]:
total_documents = len(df['content'])
min_df = 0.05 * total_documents  # Minimum document frequency to be considered
max_df = 0.8 * total_documents  # Maximum frequency allowed before filtering

In [ ]:
token_occurrences = defaultdict(int)
word_importance = defaultdict(float)

for text in tqdm(islice(df['content'], num_iterations), total=num_iterations):
    explanation = explainer.explain_instance(text, predict_proba, num_features=50, num_samples=100)
    for word, score in explanation.as_list():
        word_importance[word] += score
        token_occurrences[word] += 1

In [ ]:
# To average the scores
for word in word_importance:
    word_importance[word] /= num_iterations
    if token_occurrences[word] > 0:  # Ensure there is no division by zero
        word_importance[word] /= token_occurrences[word]

In [ ]:
adjusted_word_importance = {}
for token, importance in word_importance.items():
    occurrence = token_occurrences[token]
    if occurrence < min_df or occurrence > max_df:
        adjusted_importance = 0
    else:
        adjusted_importance = importance
    adjusted_word_importance[token] = adjusted_importance / num_iterations
    
high_word_importance = {}
for token, importance in adjusted_word_importance.items():
    high_word_importance[token] = importance*-1

In [ ]:
# Sort the words by importance
sorted_words = sorted(high_word_importance.items(), key=lambda x: x[1], reverse=True)

# Split the words and their importance scores
words, importances = zip(*sorted_words[:50])  # Top 50 words

# Create bar chart
plt.figure(figsize=(10, 8))
plt.barh(words, importances, color='skyblue')
plt.xlabel('Importance Score')
plt.title('Top 20 Word Importances')
plt.gca().invert_yaxis()  # Invert the y-axis to have the highest scores at the top
plt.show()

In [ ]:
from wordcloud import WordCloud

word_score = {word: score for word, score in sorted_words}
wordcloud = WordCloud(width = 800, height = 400, background_color ='white')

wordcloud.generate_from_frequencies(word_score)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()